In [1]:
!pip install transformers
!pip install datasets
!pip install nervaluate
!pip install accelerate -U

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from datasets import load_dataset, ClassLabel, Sequence
custom_headers = ["Id", "tags", "text"] 
dataset = load_dataset('csv', data_files={'train': ['/kaggle/input/ner-data/medical_ner.tsv']}, delimiter="\t",column_names=custom_headers)

In [4]:
print(dataset)
print(f"Number of rows in train dataset: {dataset['train'].num_rows}")

DatasetDict({
    train: Dataset({
        features: ['Id', 'tags', 'text'],
        num_rows: 49903
    })
})
Number of rows in train dataset: 49903


In [5]:
lebel_set = ['B-age', 'I-age', 'B-allergy_name', 'I-allergy_name', 'B-bmi', 'I-bmi', 'B-cancer', 'I-cancer', 'B-chronic_disease', 'I-chronic_disease', 'B-clinical_variable', 'I-clinical_variable', 'B-contraception_consent', 'I-contraception_consent', 'B-ethnicity', 'I-ethnicity', 'B-gender', 'I-gender', 'B-language_fluency', 'I-language_fluency', 'B-lower_bound', 'I-lower_bound', 'B-pregnancy', 'I-pregnancy', 'B-technology_access', 'I-technology_access', 'B-treatment', 'I-treatment', 'B-upper_bound', 'I-upper_bound', 'O']
label2id={label: i for i,label in enumerate(lebel_set)}
label2id


{'B-age': 0,
 'I-age': 1,
 'B-allergy_name': 2,
 'I-allergy_name': 3,
 'B-bmi': 4,
 'I-bmi': 5,
 'B-cancer': 6,
 'I-cancer': 7,
 'B-chronic_disease': 8,
 'I-chronic_disease': 9,
 'B-clinical_variable': 10,
 'I-clinical_variable': 11,
 'B-contraception_consent': 12,
 'I-contraception_consent': 13,
 'B-ethnicity': 14,
 'I-ethnicity': 15,
 'B-gender': 16,
 'I-gender': 17,
 'B-language_fluency': 18,
 'I-language_fluency': 19,
 'B-lower_bound': 20,
 'I-lower_bound': 21,
 'B-pregnancy': 22,
 'I-pregnancy': 23,
 'B-technology_access': 24,
 'I-technology_access': 25,
 'B-treatment': 26,
 'I-treatment': 27,
 'B-upper_bound': 28,
 'I-upper_bound': 29,
 'O': 30}

In [6]:
dataset['train'].num_rows


49903

In [7]:
import os
# Set your Hugging Face API key here
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'your_token_here'
 
# If you need to login, use the API key
from huggingface_hub import login
 
# Login using the API key
login(token=os.environ['HUGGINGFACEHUB_API_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'tags', 'text'],
        num_rows: 49903
    })
})

In [10]:
from tokenizers.pre_tokenizers import Whitespace
pre_tokenizer = Whitespace()
pre_tokenizer.pre_tokenize_str("Hello! How# are$ you? I'm% &fine, thank\" you.")

[('Hello', (0, 5)),
 ('!', (5, 6)),
 ('How', (7, 10)),
 ('#', (10, 11)),
 ('are', (12, 15)),
 ('$', (15, 16)),
 ('you', (17, 20)),
 ('?', (20, 21)),
 ('I', (22, 23)),
 ("'", (23, 24)),
 ('m', (24, 25)),
 ('%', (25, 26)),
 ('&', (27, 28)),
 ('fine', (28, 32)),
 (',', (32, 33)),
 ('thank', (34, 39)),
 ('"', (39, 40)),
 ('you', (41, 44)),
 ('.', (44, 45))]

In [11]:
from tqdm import tqdm
def get_ner(position, tags):
  all_ner_positions = list(tags.keys())
  prefix='N'
  for ner_position in all_ner_positions:
    if position[0] >= ner_position[0] and position[1] <= ner_position[1]:
      if position[0] == ner_position[0]:
        prefix = 'B-'
      else:
        prefix = 'I-'
      return  prefix+tags[ner_position]
  return 'O'

all_tokens = []
all_ner_tags = []
for idx in tqdm(range(dataset['train'].num_rows)):
  # print(dataset['train'][idx]['text'])
  position_pairs = []
  tags = {}
  tokens = []
  ner_tags = []
  for position_and_tag in dataset['train'][idx]['tags'].split(','):
    start_position, end_position, tag = int(position_and_tag.split(':')[0]), int(position_and_tag.split(':')[1]), position_and_tag.split(':')[2]
    tags[(start_position-1, end_position-1)] = tag
    position_pairs.append((start_position-1, end_position-1))
    tagged_text = dataset['train'][idx]['text'][start_position-1:end_position-1]
    # print(start_position, end_position, tag, tagged_text, dataset['train'][idx]['text'][start_position-1:end_position-1])
  # print(tags)
  # print(position_pairs)
  pre_tokenized_text = pre_tokenizer.pre_tokenize_str(dataset['train'][idx]['text'])
  for (word, position) in pre_tokenized_text:
    label = get_ner(position, tags)
    tokens.append(word)
    ner_tags.append(label2id[label])
  all_tokens.append(tokens)
  all_ner_tags.append(ner_tags)

print(len(all_tokens), len(all_ner_tags), all_tokens[100], all_ner_tags[100])

100%|██████████| 49903/49903 [00:19<00:00, 2587.58it/s]

49903 49903 ['Chronic', 'hepatobiliary', 'disease', ',', 'conservatively', 'defined', 'as', 'liver', 'function', 'tests', '(', 'AST', ',', 'ALT', ',', 'alkaline', 'phosphatase', ',', 'Total', 'Bilirubin', ')', '>', '1', '.', '5', 'times', 'the', 'upper', 'limit', 'of', 'normal'] [8, 9, 9, 30, 30, 30, 30, 30, 30, 30, 30, 10, 30, 10, 30, 10, 11, 30, 10, 11, 30, 30, 20, 21, 21, 21, 21, 21, 21, 21, 21]


In [12]:
list(tags.keys())

[(4, 12)]

In [13]:
from datasets import Dataset
my_dict = {"tokens": all_tokens, "ner_tags": all_ner_tags}
dataset = Dataset.from_dict(my_dict)

In [14]:
inputs = tokenizer(dataset[10]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'end',
 '-',
 'stage',
 'organ',
 'disease',
 'or',
 'medical',
 'condition',
 'with',
 'subsequent',
 'vision',
 'loss',
 '(',
 'e',
 '.',
 'g',
 '.',
 ',',
 'diabetes',
 ',',
 'stroke',
 ')',
 '[SEP]']

In [15]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

inputs = tokenizer(dataset[0]["tokens"], is_split_into_words=True)
inputs.tokens()

labels = dataset[0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[30, 30, 30, 30, 30, 8]
[-100, 30, 30, 30, 30, 30, 8, -100]


In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [17]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset.column_names,
)


Map:   0%|          | 0/49903 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

2024-07-07 07:10:57.930033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 07:10:57.930093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 07:10:57.931668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
!pip install seqeval
!pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [20]:
import evaluate

metric = evaluate.load("seqeval")

In [21]:
# import numpy as np
# from nervaluate import collect_named_entities
# from nervaluate import compute_metrics as ner_compute_metrics
# from nervaluate import Evaluator

# label_names = lebel_set
# def compute_metrics(eval_preds):

#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)

#     # Remove ignored index (special tokens) and convert to labels
#     true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
#     true_predictions = [
#         [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

#     true_labels_1 = [[l for l in label] for label in true_labels]
#     true_predictions_1 = [[l for l in label] for label in true_predictions]
#     evaluator = Evaluator(true_labels_1, true_predictions_1, tags=[l[2:] for l in label_names if l !='O'], loader="list")
#     results, results_by_tag = evaluator.evaluate()

#     return {
#         "precision": all_metrics["overall_precision"],
#         "recall": all_metrics["overall_recall"],
#         "f1": all_metrics["overall_f1"],
#         "accuracy": all_metrics["overall_accuracy"],
#         "results": results,
#         #"results_by_tag": results_by_tag,
#     }

In [22]:
import numpy as np
from nervaluate import collect_named_entities
from nervaluate import compute_metrics as ner_compute_metrics
from nervaluate import Evaluator

label_names = lebel_set
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute standard metrics
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    # Prepare data for evaluation
    true_labels_1 = [label for sublist in true_labels for label in sublist]
    true_predictions_1 = [label for sublist in true_predictions for label in sublist]

    # Create evaluator and compute metrics
    evaluator = Evaluator(true_labels_1, true_predictions_1, tags=[l[2:] for l in label_names if l != 'O'], loader="list")
    evaluation_results = evaluator.evaluate()

    # Depending on what evaluate() returns, you might need to adjust the unpacking
    if isinstance(evaluation_results, dict):
        results = evaluation_results.get('results')
        results_by_tag = evaluation_results.get('results_by_tag', {})
    else:
        results = evaluation_results[0]
        results_by_tag = evaluation_results[1] if len(evaluation_results) > 1 else {}

    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
        "results": results,
        "results_by_tag": results_by_tag,
    }

In [23]:
id2label = {i: label for i, label in enumerate(lebel_set)}
label2id = {v: k for k, v in id2label.items()}


In [24]:
from transformers import TrainingArguments

args = TrainingArguments(
    "clinical-BERT-FINETUNED-NER",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
tokenized_datasets= tokenized_datasets.train_test_split(test_size=0.20)

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 39922
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9981
    })
})

In [27]:
from transformers import AutoModelForTokenClassification

model_checkpoint = "medicalai/ClinicalBERT"
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
tokenized_datasets_1000 = tokenized_datasets.filter(lambda example, indice: indice <= 1000, with_indices=True)

Filter:   0%|          | 0/39922 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9981 [00:00<?, ? examples/s]

In [29]:
tokenized_datasets_1000

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1001
    })
})

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: shahzaibimtiazsatti (dsa3). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Trainer is attempting to log a value of "{'ent_type': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'partial': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'strict': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'exact': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}}" of type <class 'dict'> for key "eval/results" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'age': {'ent_type': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}

TrainOutput(global_step=1872, training_loss=0.46551627786750466, metrics={'train_runtime': 4470.2934, 'train_samples_per_second': 26.792, 'train_steps_per_second': 0.419, 'total_flos': 3189348872947896.0, 'train_loss': 0.46551627786750466, 'epoch': 3.0})

In [31]:
import os
import shutil

# Define the path of the directory or file you want to download
file_or_directory_to_save = '/kaggle/working/'

# Define the output zip file name
output_zip_file = '/kaggle/working/model.zip'

# Create a zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', file_or_directory_to_save)

print(f"Zipped file created at {output_zip_file}")


Zipped file created at /kaggle/working/model.zip


In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model_checkpoint = "/kaggle/working/clinical-BERT-FINETUNED-NER/checkpoint-1248"
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

# Define new sentences
new_sentences = [
    "His family has a history of cardiovascular diseases."
]

# Tokenize the new sentences
tokenized_inputs = tokenizer(new_sentences, truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}

# Put model in evaluation mode and move to same device as inputs
model = model.to(device)
model.eval()

# Make predictions
with torch.no_grad():
    # Perform forward pass
    outputs = model(**tokenized_inputs)

# Get the predicted token class indices
predictions = outputs.logits.argmax(dim=-1).cpu().numpy()

# Convert predicted label IDs to label names
predicted_labels = [
    [model.config.id2label[pred_id] for pred_id in sent_pred]
    for sent_pred in predictions
]

# Print results
for sent, labels in zip(new_sentences, predicted_labels):
    print(f"Sentence: {sent}")
    print(f"Predicted Labels: {labels}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence: His family has a history of cardiovascular diseases.
Predicted Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-chronic_disease', 'B-chronic_disease', 'B-chronic_disease', 'B-chronic_disease', 'I-chronic_disease', 'O', 'O']
